In [ ]:
import pandas as pd
import numpy as np

In [ ]:
swap_rates = pd.read_csv("../data/market-data-swap-rates.csv")
# swap_rates.head(50)

In [ ]:
def ema(gp):
    #  Date is in the order natrually, so no need to rearrange the date
    N = len(gp)
    alpha = 2 / (N + 1)
    coe = []
    for i in range(N):
        coe.append(alpha * ((1 - alpha) ** i))
    coe.reverse()
    # print(coe)
    return sum(np.array(gp) * np.array(coe))


In [ ]:
gp_srs = swap_rates.groupby(['Start Date', 'Tenor'])['Swap Rate'].apply(ema).reset_index()
gp_srs

KeyboardInterrupt: 

In [ ]:
swap_vols = pd.read_csv("../data/market-data-swaption-vols.csv")
swap_vols["Date"].value_counts()

2022-06-07    810
2023-12-29    810
2022-07-07    810
2021-04-13    810
2021-07-12    810
             ... 
2023-03-03    405
2022-11-30    405
2022-07-20    405
2021-12-07    405
2021-03-24    405
Name: Date, Length: 754, dtype: int64

In [ ]:
# 按照strike和expiration分组，并计算平均波动率
gp_vols = swap_vols.groupby(['Expiry', 'Tenor', 'Date'])['Vols'].mean().reset_index()
gp_vols

,Expiry,Tenor,Date,Vols
0,10y,10y,2021-01-13,0.145596
1,10y,10y,2021-01-15,0.145449
2,10y,10y,2021-01-18,0.145109
3,10y,10y,2021-01-19,0.144115
4,10y,10y,2021-01-20,0.143333
...,...,...,...,...
61069,8y,8y,2024-01-08,0.256855
61070,8y,8y,2024-01-09,0.256540
61071,8y,8y,2024-01-10,0.256868
61072,8y,8y,2024-01-11,0.258929


In [ ]:
trade_information = pd.read_csv("../data/trade-information.csv")
trade_information.head(20)

,trade name,underlying,pay_frequency,maturity,lower_bound,upper_bound
0,dummyTrade1,USD: CMS:2Y,6M,5Y,0.0042,0.0379
1,dummyTrade2,USD: CMS:2Y,4M,5Y,0.0042,0.0379
2,dummyTrade3,USD: CMS:5Y,6M,5Y,0.0044,0.0397
3,dummyTrade4,USD: CMS:5Y,4M,5Y,0.0044,0.0397
4,dummyTrade5,USD: CMS:10Y,4M,5Y,0.0046,0.0417
5,dummyTrade6,USD: CMS:10Y,3M,5Y,0.0046,0.0417
6,dummyTrade7,USD: CMS:2Y,6M,10Y,0.0042,0.0379
7,dummyTrade8,USD: CMS:2Y,4M,10Y,0.0042,0.0379
8,dummyTrade9,USD: CMS:5Y,6M,10Y,0.0044,0.0397
9,dummyTrade10,USD: CMS:5Y,4M,10Y,0.0044,0.0397


In [ ]:
vegas = pd.read_csv("../data/trade-price-ir-vegas.csv")

# Data Combination of vegas and swap rates sheet and the volities sheet

In [ ]:
vegas.head(10)

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687
5,2022-09-02,dummyTrade1,USD,0,-215402.656152,1y,2023-09-04,10y,5.016274
6,2022-09-02,dummyTrade1,USD,5,-214640.494071,1y,2023-09-04,10y,3.458172
7,2022-09-02,dummyTrade1,USD,10,-213858.452424,1y,2023-09-04,10y,3.132444
8,2022-09-02,dummyTrade1,USD,25,-211345.804723,1y,2023-09-04,10y,12.184058
9,2022-09-02,dummyTrade1,USD,50,-206384.346001,1y,2023-09-04,10y,15.648409


In [ ]:
V_vegas = pd.merge(vegas, gp_vols, left_on = ["Value Date", "Expiry Bucket", "Tenor Bucket"], \
                right_on= ["Date", "Expiry", "Tenor"], how = "left")

V_vegas = V_vegas.drop(columns = ['Expiry', "Tenor", "Date"])
V_vegas

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,Vols
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,0.223827
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,0.223827
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,0.223827
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398,0.223827
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687,0.223827
...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,-547852.055930,15y,2039-01-12,8y,5.007952,0.267817
3919100,2024-01-12,dummyTrade9,USD,25,-521228.587618,15y,2039-01-12,8y,5.131505,0.267817
3919101,2024-01-12,dummyTrade9,USD,50,-464905.982097,15y,2039-01-12,8y,3.348622,0.267817
3919102,2024-01-12,dummyTrade9,USD,75,-391315.041929,15y,2039-01-12,8y,1.063634,0.267817


In [ ]:
V_vegas2 = pd.merge(V_vegas, gp_srs, left_on = ["Value Date", "Tenor Bucket"], \
                right_on= ["Start Date", "Tenor"], how = "left")
V_vegas2.drop(columns= ["Tenor", "Start Date"])
V_vegas2

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,Vols,Start Date,Tenor,Swap Rate
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,0.223827,2022-09-02,10y,1.554418
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,0.223827,2022-09-02,10y,1.554418
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,0.223827,2022-09-02,10y,1.554418
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398,0.223827,2022-09-02,10y,1.554418
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687,0.223827,2022-09-02,10y,1.554418
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,-547852.055930,15y,2039-01-12,8y,5.007952,0.267817,2024-01-12,8y,2.044383
3919100,2024-01-12,dummyTrade9,USD,25,-521228.587618,15y,2039-01-12,8y,5.131505,0.267817,2024-01-12,8y,2.044383
3919101,2024-01-12,dummyTrade9,USD,50,-464905.982097,15y,2039-01-12,8y,3.348622,0.267817,2024-01-12,8y,2.044383
3919102,2024-01-12,dummyTrade9,USD,75,-391315.041929,15y,2039-01-12,8y,1.063634,0.267817,2024-01-12,8y,2.044383


# Date: 2022-09-02; SwapRateDifference on 09-02;

In [ ]:
# V_vegas2.to_csv("Vegas_Vol_SwapRates.csv")

# Data Combination of vegas and dummy trade information

In [ ]:
# 将pay_frequency字符串映射为相应的数值
pay_frequency_mapping = {'6M': 0.5,'4M': 0.333,'3M': 0.25}
maturity_mapping = {'5Y': 5,'10Y': 10}
trade_information['pay_frequency'] = trade_information['pay_frequency'].map(pay_frequency_mapping)
trade_information['maturity'] = trade_information['maturity'].map(maturity_mapping)
print("Transformed dummyTrade_info results:")
trade_information.head(8)

Transformed dummyTrade_info results:


,trade name,underlying,pay_frequency,maturity,lower_bound,upper_bound
0,dummyTrade1,USD: CMS:2Y,0.500,5,0.0042,0.0379
1,dummyTrade2,USD: CMS:2Y,0.333,5,0.0042,0.0379
2,dummyTrade3,USD: CMS:5Y,0.500,5,0.0044,0.0397
3,dummyTrade4,USD: CMS:5Y,0.333,5,0.0044,0.0397
4,dummyTrade5,USD: CMS:10Y,0.333,5,0.0046,0.0417
5,dummyTrade6,USD: CMS:10Y,0.250,5,0.0046,0.0417
6,dummyTrade7,USD: CMS:2Y,0.500,10,0.0042,0.0379
7,dummyTrade8,USD: CMS:2Y,0.333,10,0.0042,0.0379


underlying -> swapdifference: lower_bound upperbound --> ?

In [ ]:
merged_data = pd.merge(V_vegas2, trade_information, how='left', left_on='Trade Name', right_on='trade name')
merged_data.drop(columns=['trade name'], inplace=True)

print("Merged datasets results:")
merged_data.head(3)

# 将数据保存到新的csv文件中，文件名格式为merged_data.csv
# filename = f"merged_data.csv"
# merged_data.to_csv(filename, index=False)
# print(f"Saved {filename}")

Merged datasets results:


,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,Vols,Start Date,Tenor,Swap Rate,underlying,pay_frequency,maturity,lower_bound,upper_bound
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,0.223827,2022-09-02,10y,1.554418,USD: CMS:2Y,0.5,5,0.0042,0.0379
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,0.223827,2022-09-02,10y,1.554418,USD: CMS:2Y,0.5,5,0.0042,0.0379
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,0.223827,2022-09-02,10y,1.554418,USD: CMS:2Y,0.5,5,0.0042,0.0379


Read data：2 raw csv files

In [ ]:
# 读取数据文件
CMS_data = pd.read_csv("market-data-swap-rates.csv")
trade_info = pd.read_csv("trade-information.csv")

Pay_frequency: number of payment per year

In [ ]:
# Pay Frequency 特征架构
# 将pay_frequency字符串映射为相应的数值
pay_frequency_mapping = {
    '6M': 2,
    '4M': 3,
    '3M': 4
}
trade_info['pay_frequency'] = trade_info['pay_frequency'].map(pay_frequency_mapping)

Maturity: number of years

In [ ]:
# Maturity 特征架构
# 将maturity字符串映射为相应的数值
maturity_mapping = {
    '5Y': 5,
    '10Y': 10
}
trade_info['maturity'] = trade_info['maturity'].map(maturity_mapping)

Split market-data-swap-rate.csv: into 3 sub-files (according to underlying/tenor)

In [ ]:
# 创建underlying到Tenor的映射关系
underlying_to_tenor = {
    'USD: CMS:2Y': '2y',
    'USD: CMS:5Y': '5y',
    'USD: CMS:10Y': '10y'
}

# 创建upper_bound和lower_bound的映射关系
# 并将映射关系中的值乘以100，统一数量级以便比较
bound_info = trade_info.drop_duplicates('underlying').set_index('underlying') \
    [['upper_bound', 'lower_bound']].apply(lambda x: x * 100).to_dict('index')

# 根据underlying生成三个新的数据集
for underlying, tenor in underlying_to_tenor.items():
    filtered_cms_data = CMS_data[CMS_data['Tenor'] == tenor]
    underlying_swap_rate = f"swap-rates-{tenor}.csv"
    filtered_cms_data.to_csv(underlying_swap_rate, index=False)
    print(f"Saved {underlying_swap_rate}")

Define function: transform into features;
underlying -> CMS mean/std/max/min;
           -> interest difference daily/weekly/monthly;
lower bound & upper bound -> proportion of time within Range (based on
                             comparison between raw CMS and bounds);

In [ ]:
# Underlying Assets 特征架构
# Lower Bound & Upper Bound 特征架构

# 定义操作函数
def process_market_data(df, upper_bound, lower_bound):
# 时间序列特征：利差计算：日/周/月利差的STD
    # 转换日期格式并排序
    df['Start Date'] = pd.to_datetime(df['Start Date'])
    df = df.sort_values(by='Start Date')

    # 平均处理重复的Start Date
    df = df.groupby('Start Date').agg({'Swap Rate': 'mean'}).reset_index()

    # 填充缺失日期并插值
    full_date_range = pd.date_range(start=df['Start Date'].min(), end=df['Start Date'].max(), freq='D')
    df = df.set_index('Start Date').reindex(full_date_range).rename_axis('Start Date').reset_index()
    df['Swap Rate'] = df['Swap Rate'].interpolate()

    # 计算差分
    df['Daily Diff'] = df['Swap Rate'].diff()
    df['Weekly Diff'] = df['Swap Rate'].diff(7)
    df['Monthly Diff'] = df['Swap Rate'].diff(30)

    # 计算差分的标准差并只保留标准差的值
    daily_diff_std = df['Daily Diff'].std()
    weekly_diff_std = df['Weekly Diff'].std()
    monthly_diff_std = df['Monthly Diff'].std()

# 统计特征：CMS平均值/标准差/最大值/最小值
    # 计算均值、最大值、最小值等统计特征
    mean_cms = df['Swap Rate'].mean()
    std_cms = df['Swap Rate'].std()
    max_cms = df['Swap Rate'].max()
    min_cms = df['Swap Rate'].min()

# bound特征：利率范围内的时间比例
    # 计算利率范围内的时间比例
    df['Within Range'] = (df['Swap Rate'] >= lower_bound) & (df['Swap Rate'] <= upper_bound)
    proportion_within_range = df['Within Range'].mean()

    return (daily_diff_std, weekly_diff_std, monthly_diff_std,
            mean_cms, std_cms, max_cms, min_cms,
            proportion_within_range)


Save processed feature data: into final_results.csv

In [ ]:
# 初始化一个空的列表来存储所有数据
final_results = []

# 对每个子数据文件进行处理
for underlying, tenor in underlying_to_tenor.items():
    input_filename = f"swap-rates-{tenor}.csv"
    df = pd.read_csv(input_filename)

    # 获取upper_bound和lower_bound
    upper_bound = bound_info[underlying]['upper_bound']
    lower_bound = bound_info[underlying]['lower_bound']

    # 处理市场数据并计算统计特征和时间比例
    daily_diff_std, weekly_diff_std, monthly_diff_std, \
        mean_cms, std_cms, max_cms, min_cms, \
        proportion_within_range \
        = process_market_data(df, upper_bound, lower_bound)

    # 将结果添加到结果列表中
    final_results.append(
        {'Underlying': underlying,
         'Daily_Diff_STD': daily_diff_std,
         'Weekly_Diff_STD': weekly_diff_std,
         'Monthly_Diff_STD': monthly_diff_std,
         'Mean_CMS': mean_cms,
         'STD_CMS': std_cms,
         'Max_CMS': max_cms,
         'Min_CMS': min_cms,
         'Proportion_Within_Range': proportion_within_range})

# 将所有结果转换为DataFrame
final_df = pd.DataFrame(final_results)

# 将最终的DataFrame写入CSV文件
final_df.to_csv("final_results.csv", index=False)

Merge processed data with trade-information.csv: into final_feature_results.csv

In [ ]:
# 合并数据到trade-information.csv
final_result = pd.read_csv("final_results.csv")
merged_df = pd.merge(trade_info, final_result, how='left', left_on='underlying', right_on='Underlying')

# 删除 'Underlying' 列
merged_df.drop(columns=['Underlying'], inplace=True)

# 删除 'underlying' 列
merged_df.drop(columns=['underlying'], inplace=True)

# 将合并后的DataFrame写入CSV文件
merged_df.to_csv("final_feature_results.csv", index=False)


Merge data: V_vegas2 and trade_featureResults -> final data in one csv.

In [ ]:
trade_featureResults = pd.read_csv('final_feature_results')
merged_data = pd.merge(V_vegas2, trade_featureResults, how='left', left_on='Trade Name', right_on='trade name')
merged_data.drop(columns=['trade name'], inplace=True)

print("Merged datasets results:")
merged_data.head(3)